In [1]:
import pandas as pd
import torch
import torchtext
from torchtext.vocab import GloVe
from torchtext.data import get_tokenizer
import pytorch_lightning as pl
from torchmetrics import Accuracy, Precision, Recall, F1Score
from pytorch_lightning.loggers import TensorBoardLogger
from torch import nn

from transformers_trainer import TransformersTrainer


pl.seed_everything(42)

[rank: 0] Global seed set to 42


42

In [2]:
def run(model_name):
    acc_ckpt = pl.callbacks.ModelCheckpoint(
        monitor="avg_acc",
        mode="max",
        verbose=True,
        dirpath="../checkpoints/",
        filename=f"{model_name}",
    )

    logger = TensorBoardLogger(
        f"../logs/{model_name}", name=f"{model_name}"
    )

    model = TransformersTrainer(model_name, text_col="text")

    trainer = pl.Trainer(
        accelerator='gpu',
        precision=16,
        max_epochs=20,
        auto_select_gpus=True,
        # strategy=plugin,
        callbacks=[acc_ckpt],
        fast_dev_run=False,
        detect_anomaly=False,
        logger=logger,
    )

    trainer.fit(model)
    return trainer.test(model, ckpt_path=f'../checkpoints/{model_name}.ckpt', verbose=True)

        


In [ ]:
model_names = ['roberta-base', 'bert-base-uncased', 'distilbert-base-uncased', 'mental/mental-bert-base-uncased']
for model_name in model_names:
    torch.cuda.empty_cache()
    res = run(model_name)
    with open('res.txt', 'a') as f:
        f.writelines(f'=== {model_name} ===\n')
        for k, v in res[0].items():
            f.writelines(f'{k}: {v}\n')
        f.writelines('\n\n\n')